In [ ]:
from include.road_process import Road
from include.cool_space import CoolSpace
from include.building import Building
import geopandas as gpd
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import glob
import os

In [ ]:
# set filepaths
directory_mac = "/Volumes/T7 Shield/TUD/Synthesis/cool_place/"
directory_win = "G:\\TUD\\Synthesis\\cool_place\\"

landuse_file = directory_win + "ams_landuse_top10NL.shp"
road_file = directory_win + "ams_roads_top10NL.shp"
building_file = directory_win + "ams_buildings_bagplus.shp"
shadow_path = directory_win + "shademaps\\"

In [ ]:
# read road data
road = Road(gpd.read_file(road_file))

In [ ]:
# read building data
building = Building(gpd.read_file(building_file))

In [ ]:
# read land-use data
coolSpace = CoolSpace(gpd.read_file(landuse_file))

In [ ]:
# read all shade maps from directory
shadows = []
shadow_files = glob.glob(os.path.join(shadow_path, '*.TIF'))
for shadow_file in shadow_files:
    shadow_map = rasterio.open(shadow_file, crs=coolSpace.data.crs)
    shadows.append(shadow_map)

shadows = shadows[0:2]  # use two shade maps for testing

In [ ]:
# create road buffer
road.create_attribute('typeweg', 'buffer')
road.create_buffer('buffer')
road.data.set_geometry("buffered", inplace=True)

In [ ]:
coolSpace.clip(road.data, use_clip=False)

In [ ]:
# create building buffer and use it to clip the clipped land-use data
# the output will be the initial public space data
building.create_buffer(4)
building.data.set_geometry("buffered", inplace=True)
coolSpace.clip(building.data, use_clip=True)

In [ ]:
# perform shade calculation on public space data
coolSpace.calculate_shade(shadows, use_clip=True)

In [ ]:
# before exporting, transform the attributes contains lists into strings, so that
# it can be compatible with shapefile or geo-package format.
for i in range(len(shadows)):
    coolSpace.data[f"sdArea{i}"] = coolSpace.data[f"sdArea{i}"].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else str(x))

In [ ]:
# export all shade geometries
for i in range(len(shadows)):
    shadeGeom = coolSpace.get_shade_geometries(i)
    shadeGeom.to_file(directory_win + "shadeGeoms.gpkg", layer=f"sdGeom{i}", driver="GPKG")

In [ ]:
# export public space geometries
coolSpace_clipped = coolSpace.data.set_geometry("clipped", crs=coolSpace.data.crs)
coolSpace_clipped.drop(columns=["geometry"], inplace=True)
for i in range(len(shadows)):
    coolSpace_clipped.drop(columns=[f"sdGeom{i}"], inplace=True)
coolSpace_clipped.to_file(directory_win + "test1.shp")

In [ ]:
# export the original land-use geometries
coolSpace_origin = coolSpace.data.set_geometry("geometry")
coolSpace_origin.drop(columns=["clipped"], inplace=True)
for i in range(len(shadows)):
    coolSpace_origin.drop(columns=[f"sdGeom{i}"], inplace=True)
coolSpace_origin.to_file(directory_win + "test2.shp")

In [ ]:
# export cool space geometries
# coolSpace.evaluate_shade_coverage()
coolGeom = coolSpace.get_cool_spaces()
coolGeom.to_file(directory_win + "coolSpace.shp")